In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf

import math
import matplotlib.pyplot as plt

/Users/yueyuchen/anaconda/envs/tensorflow/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
#load data,channge type
load = np.genfromtxt("/Users/yueyuchen/Documents/ML/ML_Immune/Machine_Learning_Immunogenicity/data/bcell_peptide_nodups.csv",dtype = str, delimiter = ',')

length1 = np.shape(load)[0]
labels1 = np.zeros(length1)
load1 = np.zeros(length1)
for i in range(length1):
    labels1[i] = (load[i,1] == '1')
for i in range(length1):   
    load1[i] = len(load[i,0])
data = []
labels = []
for i in range(length1):
    if(load1[i] < 20):
        data.append(load[i,0])
        labels.append(labels1[i])

length = len(data)

Alph = {'A':1,'a':1,
        'B':2,'b':2,
        'C':3,'c':3,
        'D':4,'d':4,
        'E':5,'e':5,
        'F':6,'f':6,
        'G':7,'g':7,
        'H':8,'h':8,
        'I':9,'i':9,
        'J':10,'j':10,
        'K':11,'k':11,
        'L':12,'l':12,
        'M':13,'m':13,
        'N':14,'n':14,
        'O':15,'o':15,
        'P':16,'p':16,
        'Q':17,'q':17,
        'R':18,'r':18,
        'S':19,'s':19,
        'T':20,'t':20,
        'U':21,'u':21,
        'V':22,'v':22,
        'W':23,'w':23,
        'X':24,'x':24,
        'Y':25,'y':25,
        'Z':26,'z':26
        }
        
dataset = []
for d in data:
    d1 = []
    for l in d:
        d1.append(np.float32(Alph[l]))
    for j in range(20-len(d1)):
        d1.append(np.float32(0))
    dataset.append(d1)

for i in range(len(labels)):
    if (labels[i] == 0):
        labels[i] = [1,0]
    else:
        labels[i] = [0,1]
        
for i in range(length):
    dataset[i] = np.asarray(dataset[i])
    labels[i] = np.float32(np.asarray(labels[i]))

dataset = np.asarray(dataset)
labels = np.asarray(labels)



In [3]:
datamax = np.amax(dataset,axis = 0)
datamin = np.amin(dataset,axis = 0)
datarange = datamax - datamin
for i in range(len(datarange)):
    if(datarange[i] == 0):
        datarange[i] = 1
dataavg = np.mean(dataset,axis = 0)
for i in range(len(dataset)):
    dataset[i] = (dataset[i]-dataavg)/datarange

In [4]:
#shuffle dataset
shuf = np.append(dataset,labels,axis = 1)
np.random.shuffle(shuf)

dataset = shuf[:,:20]
labels = shuf[:,20:22]
print (np.shape(dataset),np.shape(labels))

(237603, 20) (237603, 2)


In [5]:
trainnum = 150000
testnum = 10000

train_dataset = dataset[:trainnum]
train_labels = labels[:trainnum]

test_dataset = dataset[trainnum:trainnum + testnum]
test_labels = labels[trainnum :trainnum + testnum]



valid_dataset = dataset[trainnum + testnum:trainnum + 2*testnum]
valid_labels = labels[trainnum + testnum:trainnum + 2*testnum]

In [6]:
batch_size = 1000
num_hidden_nodes = 20
peplength = 20
num_labels = 2

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, peplength))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_test_dataset = tf.placeholder(tf.float32,
                                    shape=(testnum, peplength))
    tf_test_labels = tf.placeholder(tf.float32, shape=(testnum, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
     
    beta_regul = tf.placeholder(tf.float32)
  
  # Variables.
    weights1 = tf.Variable(
      tf.truncated_normal([peplength, num_hidden_nodes]))
    biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    weights2 = tf.Variable(
      tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
    lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    tlogits = tf.matmul(lay1_train, weights2) + biases2
    
    lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    vlogits = tf.matmul(lay1_valid, weights2) + biases2
    
    tloss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(tlogits, tf_train_labels))+beta_regul * (tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2))
    
    vloss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(vlogits, valid_labels))+beta_regul * (tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2))
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.0019).minimize(tloss)
  
  # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(tlogits)
    
    valid_prediction = tf.nn.softmax(tf.matmul(lay1_valid, weights2) + biases2)
    lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(lay1_test, weights2) + biases2)

In [7]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [8]:
num_steps = 30000
train_curv = np.zeros(0)
valid_curv = np.zeros(0)
test_result = np.zeros(0)

tloss_curv = np.zeros(0)
vloss_curv = np.zeros(0)

with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        
        
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, 
                     tf_test_dataset: test_dataset,tf_test_labels:test_labels,beta_regul: 1e-3}
        _, tl, vl, predictions = session.run(
          [optimizer, tloss, vloss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, tl))
            ac = accuracy(predictions, batch_labels)
            train_curv = np.append(train_curv,ac)
            print("Minibatch accuracy: %.1f%%" % ac)
            ac = accuracy(valid_prediction.eval(), valid_labels)
            valid_curv = np.append(valid_curv,ac)
            print("Validation accuracy: %.1f%%" % ac)
            tloss_curv = np.append(tloss_curv,tl)
            vloss_curv = np.append(vloss_curv,vl)
        
    for i in range(15):
        test_dataset = dataset[i*testnum:i*testnum + testnum]
        test_labels = labels[i*testnum:i*testnum + testnum]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, 
                     tf_test_dataset: test_dataset, tf_test_labels:test_labels, beta_regul: 1e-3}
        testpre = session.run(
          test_prediction, feed_dict=feed_dict)
        ac = accuracy(testpre,test_labels)
        test_result = np.append(test_result,ac)
    

Initialized
Minibatch loss at step 0: 1.065276
Minibatch accuracy: 68.2%
Validation accuracy: 67.0%
Minibatch loss at step 500: 0.797142
Minibatch accuracy: 81.8%
Validation accuracy: 82.3%
Minibatch loss at step 1000: 0.701431
Minibatch accuracy: 85.8%
Validation accuracy: 85.9%
Minibatch loss at step 1500: 0.520804
Minibatch accuracy: 89.9%
Validation accuracy: 87.1%
Minibatch loss at step 2000: 0.570494
Minibatch accuracy: 88.2%
Validation accuracy: 87.6%
Minibatch loss at step 2500: 0.528494
Minibatch accuracy: 88.6%
Validation accuracy: 88.1%
Minibatch loss at step 3000: 0.525758
Minibatch accuracy: 90.6%
Validation accuracy: 88.3%
Minibatch loss at step 3500: 0.586460
Minibatch accuracy: 86.5%
Validation accuracy: 88.5%
Minibatch loss at step 4000: 0.499962
Minibatch accuracy: 90.6%
Validation accuracy: 88.7%
Minibatch loss at step 4500: 0.500110
Minibatch accuracy: 89.4%
Validation accuracy: 88.9%
Minibatch loss at step 5000: 0.496346
Minibatch accuracy: 89.5%
Validation accurac

In [41]:
c = np.arange(60)
#plt.plot(c,nodevsperformance[0],'r--',c,nodevsperformance[1],'r--',
#        c,nodevsperformance[5],'bs',c,nodevsperformance[6],'bs',
 #       c,nodevsperformance[10],'g^',c,nodevsperformance[11],'g^')
plt.plot(c,nodevsperformance[2],'r--',label = 'train&validation accuracy(20nodes)')
plt.plot(c,nodevsperformance[3],'r--')

plt.plot(c,nodevsperformance[7],'bs',label = 'train&validation accuracy(50nodes)')
plt.plot(c,nodevsperformance[8],'bs')

plt.plot(c,nodevsperformance[12],'g^',label = 'train&validation accuracy(100nodes)')
plt.plot(c,nodevsperformance[13],'g^')


plt.xlabel('iterations')
plt.ylabel('prediction accuracy')
plt.title('prediction accuracy vs NN nodes')
plt.legend(loc = 'best')
plt.show()


In [9]:
test_result


array([ 89.7 ,  90.01,  90.02,  89.78,  90.39,  90.6 ,  90.4 ,  89.96,
        90.74,  90.13,  90.46,  90.3 ,  90.46,  90.78,  89.89])

In [23]:
nodenum = []
nodevsperformance = []

In [37]:

nodenum.append(50)
nodevsperformance.append(tloss_curv)
nodevsperformance.append(vloss_curv)
nodevsperformance.append(train_curv)
nodevsperformance.append(valid_curv)
nodevsperformance.append(test_result)


In [25]:
np.shape(tloss_curv)

(60,)

In [38]:
np.save('/Users/yueyuchen/Documents/ML/ML_Immune/Machine_Learning_Immunogenicity/data/nodedatanew',nodevsperformance)